# Wines Points prediction 

Submission Date : 3.6.2023
Task: Predict the wine score given the inputs
Instructions:
 * Use logistic regression as benchmark model
 * Use sklearn pipeliens + cv + grid search with sklearn models (e.g. KNNs, RandomForest, etc.)
 * Compare all models on proper metric (your choice)

For DNN course project:
* Use sklearn pipeliens with tensorflow models (w/wo embeddings, LSTMs, RNNs, Transformers etc.)
* Compare all models on proper metric (your choice)

In [3]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [4]:
import pandas as pd
import cufflinks as cf; cf.go_offline()

In [5]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate, Flatten, TextVectorization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical, plot_model, pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import activations

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


# Embedding

In [131]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.layers import Input, Embedding, LSTM, Concatenate, Dense
from tensorflow.keras.models import Model

# Load your dataset
wine_reviews = pd.read_csv("clean_wine_reviews_data.csv")
summary_df = pd.read_csv("summary_df.csv")

# Preprocessing
wine_reviews = wine_reviews.drop(['Unnamed: 0', 'taster_twitter_handle', 'region_1', 'region_2', 'designation'], axis=1)
wine_reviews['description_length'] = wine_reviews.description.str.len()
wine_reviews = wine_reviews.dropna(subset=['year', 'points'])
wine_reviews['year'] = wine_reviews['year'].astype(int).astype(str)
wine_reviews["description_length"] = wine_reviews["description_length"].astype(np.int32)

# Split data
y = wine_reviews['points']
X = wine_reviews.drop(['points'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=29)

# Preprocess categorical data
categorical_cols = ['country', 'taster_name', 'variety', 'province', 'year']
numeric_cols = ['price']

# Use StandardScaler for numeric columns
numeric_transformer = StandardScaler()

# Use OneHotEncoder for categorical columns
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse=False)

# Preprocess features using ColumnTransformer
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Fit and transform preprocessing on training data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Text Vectorization
max_vocab_length = 10000
max_length = 60

text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length)

text_vectorizer.adapt(X_train['description'])
text_input = Input(shape=(1,), dtype="string")
text_vector = text_vectorizer(text_input)
text_embedding = Embedding(
    input_dim=max_vocab_length,
    output_dim=128,
    embeddings_initializer="uniform",
    input_length=max_length)(text_vector)
text_lstm = LSTM(64)(text_embedding)

# Combine numeric and categorical data
combined_input = Input(shape=(X_train_transformed.shape[1],), dtype="float32")
concatenated = Concatenate()([text_lstm, combined_input])

# Build the model
x = Dense(16, activation='relu')(concatenated)
x = Dense(8, activation='relu')(x)
output = Dense(1, activation="linear")(x)

model = Model(inputs=[text_input, combined_input], outputs=output)

# Compile the model
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mse'])

# Train the model
model.fit([X_train['description'], X_train_transformed], y_train,
          batch_size=64,
          epochs=10,
          validation_split=0.2) 



C:\Users\lior\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Epoch 1/10
1012/1012 [==============================] - 80s 74ms/step - loss: 467.8390 - mse: 467.8390 - val_loss: 6.1965 - val_mse: 6.1965
Epoch 2/10
1012/1012 [==============================] - 137s 135ms/step - loss: 5.9820 - mse: 5.9820 - val_loss: 5.8643 - val_mse: 5.8643
Epoch 3/10
1012/1012 [==============================] - 81s 80ms/step - loss: 5.8069 - mse: 5.8069 - val_loss: 5.8129 - val_mse: 5.8129
Epoch 4/10
1012/1012 [==============================] - 73s 72ms/step - loss: 5.7820 - mse: 5.7820 - val_loss: 5.7795 - val_mse: 5.7795
Epoch 5/10
1012/1012 [==============================] - 93s 92ms/step - loss: 5.7959 - mse: 5.7959 - val_loss: 5.7881 - val_mse: 5.7881
Epoch 6/10
1012/1012 [==============================] - 151s 149ms/step - loss: 5.8024 - mse: 5.8024 - val_loss: 6.0474 - val_mse: 6.0474
Epoch 7/10
1012/1012 [==============================] - 129s 128ms/step - loss: 5.8364 - mse: 5.8364 - val_loss: 6.2460 - val_mse: 6.2460
Epoch 8/10
1012/1012 [================

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

# Load your dataset
wine_reviews = pd.read_csv("clean_wine_reviews_data.csv")

            
# Preprocessing
wine_reviews = wine_reviews.drop(['Unnamed: 0', 'taster_twitter_handle', 'region_1', 'region_2', 'designation'], axis=1)
wine_reviews['description_length'] = wine_reviews.description.str.len()
wine_reviews = wine_reviews.dropna(subset=['year', 'points'])
wine_reviews['year'] = wine_reviews['year'].astype(int).astype(str)
wine_reviews["description_length"] = wine_reviews["description_length"].astype(np.int32)

# Split data
y = wine_reviews['points']
X = wine_reviews.drop(['points'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=29)

# Define a range of hyperparameters to search over
first_layer_list = [16, 32]
second_layer_list = [8, 16]
learning_rate_list = [0.001, 0.01]

best_model = None
best_score = float('inf')

for first_layer in first_layer_list:
    for second_layer in second_layer_list:
        for lrn_rate in learning_rate_list:
            print(f"Training model with {first_layer} layers, {second_layer} units per layer, and learning rate {lrn_rate}")


            # Preprocess categorical data
            categorical_cols = ['country', 'taster_name', 'variety', 'province', 'year']
            numeric_cols = ['price']

            # Use StandardScaler for numeric columns
            numeric_transformer = StandardScaler()

            # Use OneHotEncoder for categorical columns
            categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse=False)

            # Preprocess features using ColumnTransformer
            from sklearn.compose import ColumnTransformer

            preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, numeric_cols),
                    ('cat', categorical_transformer, categorical_cols)
                ])

            # Fit and transform preprocessing on training data
            X_train_transformed = preprocessor.fit_transform(X_train)
            X_test_transformed = preprocessor.transform(X_test)

            # Text Vectorization
            max_vocab_length = 10000
            max_length = 60

            text_vectorizer = tf.keras.layers.TextVectorization(
                max_tokens=max_vocab_length,
                output_mode="int",
                output_sequence_length=max_length)

            text_vectorizer.adapt(X_train['description'])
            text_input = Input(shape=(1,), dtype="string")
            text_vector = text_vectorizer(text_input)
            text_embedding = Embedding(
                input_dim=max_vocab_length,
                output_dim=128,
                embeddings_initializer="uniform",
                input_length=max_length)(text_vector)
            text_lstm = LSTM(64)(text_embedding)

            # Combine numeric and categorical data
            combined_input = Input(shape=(X_train_transformed.shape[1],), dtype="float32")
            concatenated = Concatenate()([text_lstm, combined_input])

            # Build the model
            x = Dense(first_layer, activation='relu')(concatenated)
            x = Dense(second_layer, activation='relu')(x)
            output = Dense(1, activation="linear")(x)
            
    
            # Define the learning rate variable
            learning_rate = tf.Variable(lrn_rate, trainable=False)
            tf.keras.backend.set_value(learning_rate, lrn_rate)

            model = Model(inputs=[text_input, combined_input], outputs=output)

            # Compile the model
            model.compile(loss=tf.keras.losses.MeanSquaredError(),
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mse'])

            # Train the model
            model.fit([X_train['description'], X_train_transformed], y_train,
                      batch_size=64,
                      epochs=10,
                      validation_split=0.2) 


            loss, mse = model.evaluate([X_test['description'], X_test_transformed], y_test, verbose=0)

            # Check if this is the best model so far
            if mse < best_score:
                best_score = mse
                best_model = model
                

predictions = best_model.predict([X_test['description'], X_test_transformed])


Training model with 16 layers, 8 units per layer, and learning rate 0.001


C:\Users\lior\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Epoch 1/10
1012/1012 [==============================] - 73s 68ms/step - loss: 744.4686 - mse: 744.4686 - val_loss: 6.2972 - val_mse: 6.2972
Epoch 2/10
1012/1012 [==============================] - 70s 69ms/step - loss: 6.0590 - mse: 6.0590 - val_loss: 5.9055 - val_mse: 5.9055
Epoch 3/10
1012/1012 [==============================] - 69s 68ms/step - loss: 5.8431 - mse: 5.8431 - val_loss: 5.8243 - val_mse: 5.8243
Epoch 4/10
1012/1012 [==============================] - 69s 68ms/step - loss: 5.7957 - mse: 5.7957 - val_loss: 5.8830 - val_mse: 5.8830
Epoch 5/10
1012/1012 [==============================] - 69s 68ms/step - loss: 5.7864 - mse: 5.7864 - val_loss: 5.7897 - val_mse: 5.7897
Epoch 6/10
1012/1012 [==============================] - 69s 68ms/step - loss: 5.7934 - mse: 5.7934 - val_loss: 5.7667 - val_mse: 5.7667
Epoch 7/10
1012/1012 [==============================] - 69s 68ms/step - loss: 5.8043 - mse: 5.8043 - val_loss: 5.8434 - val_mse: 5.8434
Epoch 8/10
1012/1012 [======================

C:\Users\lior\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Epoch 1/10
1012/1012 [==============================] - 71s 67ms/step - loss: 747.8009 - mse: 747.8009 - val_loss: 6.2652 - val_mse: 6.2652
Epoch 2/10
1012/1012 [==============================] - 68s 67ms/step - loss: 6.0320 - mse: 6.0320 - val_loss: 5.8923 - val_mse: 5.8923
Epoch 3/10
1012/1012 [==============================] - 67s 66ms/step - loss: 5.8326 - mse: 5.8326 - val_loss: 5.8514 - val_mse: 5.8514
Epoch 4/10
1012/1012 [==============================] - 68s 67ms/step - loss: 5.7841 - mse: 5.7841 - val_loss: 5.7778 - val_mse: 5.7778
Epoch 5/10
1012/1012 [==============================] - 67s 66ms/step - loss: 5.7798 - mse: 5.7798 - val_loss: 5.8106 - val_mse: 5.8106
Epoch 6/10
1012/1012 [==============================] - 68s 67ms/step - loss: 5.7873 - mse: 5.7873 - val_loss: 5.7989 - val_mse: 5.7989
Epoch 7/10
1012/1012 [==============================] - 67s 66ms/step - loss: 5.8161 - mse: 5.8161 - val_loss: 5.7759 - val_mse: 5.7759
Epoch 8/10
1012/1012 [======================

...

C:\Users\lior\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Epoch 1/10
1012/1012 [==============================] - 80s 75ms/step - loss: 273.4088 - mse: 273.4088 - val_loss: 5.8429 - val_mse: 5.8429
Epoch 2/10
1012/1012 [==============================] - 76s 75ms/step - loss: 5.8388 - mse: 5.8388 - val_loss: 5.9686 - val_mse: 5.9686
Epoch 3/10
1012/1012 [==============================] - 76s 75ms/step - loss: 5.8713 - mse: 5.8713 - val_loss: 5.8381 - val_mse: 5.8381
Epoch 4/10
1012/1012 [==============================] - 77s 76ms/step - loss: 5.9346 - mse: 5.9346 - val_loss: 5.8894 - val_mse: 5.8894
Epoch 5/10
1012/1012 [==============================] - 76s 75ms/step - loss: 6.0305 - mse: 6.0305 - val_loss: 7.7384 - val_mse: 7.7384
Epoch 6/10
1012/1012 [==============================] - 77s 76ms/step - loss: 6.0413 - mse: 6.0413 - val_loss: 6.7371 - val_mse: 6.7371
Epoch 7/10
1012/1012 [==============================] - 77s 77ms/step - loss: 6.0892 - mse: 6.0892 - val_loss: 6.1751 - val_mse: 6.1751
Epoch 8/10
1012/1012 [======================

C:\Users\lior\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



Epoch 1/10
1012/1012 [==============================] - 90s 85ms/step - loss: 281.2530 - mse: 281.2530 - val_loss: 5.8312 - val_mse: 5.8312
Epoch 2/10
1012/1012 [==============================] - 90s 89ms/step - loss: 5.8566 - mse: 5.8566 - val_loss: 5.9373 - val_mse: 5.9373
Epoch 3/10
1012/1012 [==============================] - 94s 92ms/step - loss: 5.8946 - mse: 5.8946 - val_loss: 5.8917 - val_mse: 5.8917
Epoch 4/10
1012/1012 [==============================] - 100s 99ms/step - loss: 5.9632 - mse: 5.9632 - val_loss: 5.8195 - val_mse: 5.8195
Epoch 5/10
1012/1012 [==============================] - 96s 95ms/step - loss: 5.9857 - mse: 5.9857 - val_loss: 6.3758 - val_mse: 6.3758
Epoch 6/10
1012/1012 [==============================] - 87s 86ms/step - loss: 6.0533 - mse: 6.0533 - val_loss: 6.1915 - val_mse: 6.1915
Epoch 7/10
1012/1012 [==============================] - 87s 86ms/step - loss: 6.0673 - mse: 6.0673 - val_loss: 5.8530 - val_mse: 5.8530
Epoch 8/10
1012/1012 [=====================

NameError: name 'best_params' is not defined

In [7]:
predictions = best_model.predict([X_test['description'], X_test_transformed])

843/843 [==============================] - 17s 20ms/step


In [8]:
preds = tf.squeeze(tf.round(predictions))
print(r2_score(y_test, preds))

0.3622686397744588


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.metrics import MeanSquaredError
early_stopping_monitor = EarlyStopping(
    monitor='mean_squared_error',
    min_delta=0.01,
    patience=3,
    verbose=0,
    mode='min',
    baseline=None,
    restore_best_weights=True
) 

model.compile(optimizer=Adam(),
              loss='mean_squared_error',
              metrics=Accuracy())
# Train the model
model.fit( [X_train['description'], X_train_transformed], y_train,
          batch_size=16,
          epochs=10,
          callbacks=[early_stopping_monitor])

# Evaluate the model
accuracy = model.evaluate([X_test['description'], X_test_transformed], y_test, verbose=0)

Epoch 1/10
5057/5057 [==============================] - 216s 42ms/step - loss: 5.5139 - accuracy: 0.0000e+00 - mean_squared_error: 0.0000e+00
Epoch 2/10
5057/5057 [==============================] - 208s 41ms/step - loss: 5.5124 - accuracy: 1.2359e-05 - mean_squared_error: 0.0000e+00
Epoch 3/10
5057/5057 [==============================] - 210s 42ms/step - loss: 5.4933 - accuracy: 0.0000e+00 - mean_squared_error: 0.0000e+00
Epoch 4/10
1740/5057 [=========>....................] - ETA: 2:18 - loss: 5.4747 - accuracy: 0.0000e+00 - mean_squared_error: 0.0000e+00

In [ ]:
predictions = model.predict([X_test['description'], X_test_transformed])
preds = tf.squeeze(tf.round(predictions))
print(r2_score(y_test, preds))

In [ ]:
print(loss, accuracy)